# Анализ данных о продажах игра для интернет-магазина «Стримчик» 

Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.
В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортируем библиотеки для работы
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as st
from numpy import nan

In [2]:
#читаем файл
try:
    data = pd.read_csv('C:/Users/MiPro/Desktop/python/games.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')    

In [3]:
#посмотрим на данные
data.head(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [4]:
#переименуем столбцы
data.rename(columns = {'Name' : 'name', 'Platform' : 'platform', 'Year_of_Release' : 'year_of_release', 
                    'Genre' : 'genre', 'NA_sales' : 'na_sales', 'EU_sales' : 'eu_sales', 
                     'JP_sales' : 'jp_sales', 'Other_sales' : 'other_sales', 'Critic_Score' : 'critic_score', 
                     'Rating' : 'rating', 'User_Score' : 'user_score'                   
                    }, inplace = True)

In [10]:
#посмотрим что за переменные
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16445 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


In [6]:
#немного есть ошибки в именнах, поправили. регистр менять нет смысла
unique_name = data.sort_values('name')['name'].unique()
for i in unique_name:
    print(i)
  
for i in range(len(data['name'])):
    if data.loc[i,'name'] == ' Beyblade Burst':
        data.loc[i,'name'] = 'Beyblade Burst'
    if data.loc[i,'name'] == ' Fire Emblem Fates':
        data.loc[i,'name'] = 'Fire Emblem Fates'
    if data.loc[i,'name'] == " Frozen: Olaf's Quest":
        data.loc[i,'name'] = "Frozen: Olaf's Quest"
    if data.loc[i,'name'] == ' Haikyu!! Cross Team Match!':
        data.loc[i,'name'] = 'Haikyu!! Cross Team Match!'
    if data.loc[i,'name'] == ' Tales of Xillia 2':
        data.loc[i,'name'] = 'Tales of Xillia 2'
    if data.loc[i,'name'] == "'98 Koshien":
        data.loc[i,'name'] = '98 Koshien'


 Beyblade Burst
 Fire Emblem Fates
 Frozen: Olaf's Quest
 Haikyu!! Cross Team Match!
 Tales of Xillia 2
'98 Koshien
.hack//G.U. Vol.1//Rebirth
.hack//G.U. Vol.2//Reminisce
.hack//G.U. Vol.2//Reminisce (jp sales)
.hack//G.U. Vol.3//Redemption
.hack//Infection Part 1
.hack//Link
.hack//Mutation Part 2
.hack//Outbreak Part 3
.hack//Quarantine Part 4: The Final Chapter
.hack: Sekai no Mukou ni + Versus
007 Racing
007: Quantum of Solace
007: The World is not Enough
007: Tomorrow Never Dies
1 vs. 100
1/2 Summer +
10 Minute Solution
100 All-Time Favorites
100 Classic Books
100 Classic Games
1000 Cooking Recipes from ELLE à table
1001 Touch Games
101-in-1 Explosive Megamix
101-in-1 Party Megamix Wii
101-in-1 Sports Megamix
101-in-1 Sports Party Megamix
1080°: TenEighty Snowboarding
11eyes: CrossOver
12-Sai. Honto no Kimochi
12-Sai. Koisuru Diary
12Riven: The Psi-Climinal of Integral
15 Days
1701 A.D.
18 Wheeler: American Pro Trucker
18 Wheels of Steel: Extreme Trucker
18 Wheels of Steel: Extre

Cartoon Network: Punch Time Explosion
Cartoon Network: Punch Time Explosion XL
Carve
Case Closed: One Truth Prevails
Casino Chaos With Las Vegas Players Collection
Casper
Casper's Scare School: Classroom Capers
Casper's Scare School: Spooky Sports Day
Casper: Friends Around the World
Casper: Spirit Dimensions
Castle Shikigami 2
Castle of Shikigami III
Castlevania
Castlevania Bloodlines
Castlevania Chronicles
Castlevania Double Pack
Castlevania Dracula X
Castlevania II: Simon's Quest
Castlevania III: Dracula's Curse
Castlevania Judgment
Castlevania: Aria of Sorrow
Castlevania: Circle of the Moon
Castlevania: Curse of Darkness
Castlevania: Dawn of Sorrow
Castlevania: Harmony of Dissonance
Castlevania: Lament of Innocence
Castlevania: Legacy of Darkness
Castlevania: Lords of Shadow
Castlevania: Lords of Shadow - Mirror of Fate
Castlevania: Lords of Shadow - Reverie
Castlevania: Lords of Shadow 2
Castlevania: Order of Ecclesia
Castlevania: Portrait of Ruin
Castlevania: Symphony of the Nigh

Cursed Mountain
Custer's Revenge
Custom Beat Battle: Draglade 2
Custom Drive
Custom Robo
Custom Robo Arena
Custom Robo V2
Cut the Rope
Cy Girls
Cyber Sled
Cyber Troopers Virtual-On
Cyber Troopers Virtual-On Marz
CyberTiger
Cyberia
D
D.C. II P.S.: Da Capo II Plus Situation
D.C. III: Da Capo III
D.C.F.S.: Da Capo Four Seasons
D.C.I.F.: Da Capo Innocent Finale
D.Gray-man: Kami no Shitotachi
D.Gray-man: Sousha no Shikaku
D1 Professional Drift Grand Prix Series
DC Universe Online
DDRMAX2: Dance Dance Revolution
DEATH NOTE: Kira Game
DEATH NOTE: L o Tsugu Mono
DECA Sports Freedom
DICE: DNA Integrated Cybernetic Enterprises
DJ Hero
DJ Hero 2
DJ Max Fever
DJ Max Portable 3
DJ Max Technika Tune
DJ Star
DK: King of Swing
DOA 2: Dead or Alive 2 Hardcore
DS Bimoji Training
DS Bungaku Zenshuu
DS Kageyama Method: Dennou Hanpuku - Masu x Masu Hyaku Masu Keisan
DS Kageyama Method: Dennou Hanpuku - Tadashii Kanji Kakitori-Kun
DS Nishimura Kyotaro Suspense 2 Shin Tantei Series: Kanazawa Hakodate - Gokka

Final Fantasy IX
Final Fantasy Legend III
Final Fantasy Tactics
Final Fantasy Tactics A2: Grimoire of the Rift
Final Fantasy Tactics Advance
Final Fantasy Tactics: The War of the Lions
Final Fantasy Type-0
Final Fantasy V
Final Fantasy V Advance
Final Fantasy VI
Final Fantasy VI Advance
Final Fantasy VII
Final Fantasy VII International
Final Fantasy VIII
Final Fantasy X
Final Fantasy X / X-2 HD Remaster
Final Fantasy X International
Final Fantasy X-2
Final Fantasy X-2: International + Last Mission
Final Fantasy X/X-2 HD Remaster
Final Fantasy XI
Final Fantasy XI: All-In-One Pack 2006
Final Fantasy XI: Chains of Promathia
Final Fantasy XI: Rise of the Zilart
Final Fantasy XI: Treasures of Aht Urhgan
Final Fantasy XI: Ultimate Collection
Final Fantasy XI: Vana'diel Collection 2008
Final Fantasy XI: Wings of the Goddess
Final Fantasy XII
Final Fantasy XII International Zodiac Job System
Final Fantasy XII: Revenant Wings
Final Fantasy XIII
Final Fantasy XIII-2
Final Fantasy XIV: A Realm Re

Grabbed by the Ghoulies
Gradius
Gradius Collection
Gradius Galaxies
Gradius III and IV
Gradius V
Graffiti Kingdom
Gran Turismo
Gran Turismo (PSP)
Gran Turismo 2
Gran Turismo 3: A-Spec
Gran Turismo 4
Gran Turismo 4 Prologue
Gran Turismo 5
Gran Turismo 5 Prologue
Gran Turismo 6
Gran Turismo Concept 2001 Tokyo
Grand Ages: Medieval
Grand Kingdom
Grand Knights History
Grand Prix
Grand Prix Challenge
Grand Prix Legends
Grand Slam
Grand Slam Tennis 2
Grand Theft Auto
Grand Theft Auto 2
Grand Theft Auto III
Grand Theft Auto IV
Grand Theft Auto V
Grand Theft Auto: Chinatown Wars
Grand Theft Auto: Liberty City Stories
Grand Theft Auto: Mission Pack #1, London 1969
Grand Theft Auto: San Andreas
Grand Theft Auto: Vice City
Grand Theft Auto: Vice City Stories
Grandia
Grandia II
Grandia III
Grandia Xtreme
Gravitar
Gravity Falls: Legend of the Gnome Gemulets
Gravity Games Bike: Street Vert Dirt
Gravity Rush
Gravity Rush Remastered
Gray Matter
Grease
Grease Dance
Great Battle Full Blast
Great Edo Blac

...

Pro Yakyuu Family Stadium
Pro Yakyuu Family Stadium '87
Pro Yakyuu Family Stadium '88
Pro Yakyuu Famista 2011
Pro Yakyuu Famista DS
Pro Yakyuu Famista DS 2009
Pro Yakyuu Famista DS 2010
Pro Yakyuu Greatest Nine '97
Pro Yakyuu Greatest Nine 98
Pro Yakyuu Netsu Star 2006
Pro Yakyuu Netsu Star 2007
Pro Yakyuu Spirits 5 Kanzenban
Pro Yakyuu Team o Tsukurou!
Pro Yakyuu Team o Tsukurou! 2
Professional Farmer 2016
Professional Fisherman's Tour: Northern Hemisphere
Professor Heinz Wolff's Gravity
Professor Layton and the Azran Legacy
Professor Layton and the Curious Village
Professor Layton and the Diabolical Box
Professor Layton and the Last Specter
Professor Layton and the Mask of Miracle
Professor Layton and the Unwound Future
Professor Layton vs Phoenix Wright: Ace Attorney
Project CARS
Project Eden
Project FIFA World Cup
Project Gotham Racing
Project Gotham Racing (JP weekly sales)
Project Gotham Racing 2 (JP weekly sales)
Project Hacker: Kakusei
Project Overkill
Project Runway
Project Se

Spore Hero
Spore Hero Arena
Sports Car GT
Sports Champions
Sports Collection
Sports Illustrated for Kids: Baseball
Sports Illustrated for Kids: Football
Spot Goes to Hollywood
Sprint Cars: Road to Knoxville
Sprung - A Game Where Everyone Scores
Spy Fiction
Spy Fox in Dry Cereal
Spy Games: Elevator Mission
Spy Hunter
Spy Hunter / Super Sprint
Spy Hunter 2
Spy Hunter: Nowhere To Run
Spy Kids 3-D: Game Over
Spy Kids Challenger
Spy vs Spy
Spyborgs
Spyro 2: Ripto's Rage!
Spyro 2: Season of Flame
Spyro Orange: The Cortex Conspiracy
Spyro the Dragon
Spyro: A Hero's Tail
Spyro: Attack of the Rhynocs
Spyro: Enter the Dragonfly
Spyro: Season of Ice
Spyro: Shadow Legacy
Spyro: Year of the Dragon
Squeeballs Party
Squinkies
Squinkies 2: Adventure Mall Surprize!
Squishy Tank
Stacked with Daniel Negreanu
Stadium Games
Stake: Fortune Fighters
Star Driver: Kagayaki no Takuto - Ginga Bishounen Densetsu
Star Fighter
Star Fox
Star Fox 64
Star Fox 64 3D
Star Fox Adventures
Star Fox Command
Star Fox: Assaul

Suzumiya Haruhi-Chan no Mahjong
Suzunone Seven: Rebirth Knot
Sweet Honey Coming
Swing Away Golf
Swingerz Golf
Sword & Sorcery
Sword Art Online: Game Director's Edition
Sword Art Online: Hollow Fragment
Sword Art Online: Hollow Realization
Sword Art Online: Infinity Moment
Sword Art Online: Lost Song
Sword of Mana
Sword of the Samurai
Swords
Syberia
Syberia II
Sydney 2000
Syndicate
Syndicate Wars
Syphon Filter
Syphon Filter 2
Syphon Filter 3
Syphon Filter: Dark Mirror
Syphon Filter: Logan's Shadow
Syphon Filter: The Omega Strain
System 3 presents Ferrari Challenge Trofeo Pirelli
System Flaw
Sébastien Loeb Rally Evo
T'ai Fu: Wrath of the Tiger
T.A.C. Heroes : Big Red One
T.R.A.G. - Tactical Rescue Assault Group: Mission of Mercy
TERA
TMNT
TMNT: Mutant Melee
TNA iMPACT!
TNA iMPACT: Cross the Line
TNN Motor Sports Hardcore 4x4
TOCA 2: Touring Cars
TOCA Race Driver 2
TOCA Race Driver 2: Ultimate Racing Simulator
TOCA Race Driver 3
TOCA Touring Car Championship
TOEIC Test Training DS
TRINITY

Watch Dogs
Watch Dogs 2
Watchmen: The End is Nigh - The Complete Experience
Watchmen: The End is Nigh Part 1 & 2
Water Sports
Wave Race 64
Wave Race: Blue Storm
Wave Rally
Way of the Samurai
Way of the Samurai 2
Way of the Samurai 2 Portable
Way of the Samurai 3
Way of the Samurai 4
Wayne Gretzky's 3D Hockey
Wayne Gretzky's 3D Hockey '98
We Cheer
We Cheer 2
We Dance
We Love Golf!
We Love Katamari
We Sing
We Sing Deutsche Hits
We Sing Encore
We Sing Pop!
We Sing Robbie Williams
We Sing Rock!
We Sing UK Hits
We Ski
We Ski & Snowboard
We Wish You A Merry Christmas
Wedding Dash
Weiss Schwarz Portable
Western Riding Academy
Wet
Wetrix
Whac-A-Mole
Whacked!
What Did I Do to Deserve This, My Lord!? 2
What's Cooking? Jamie Oliver
Wheel of Fortune
Wheel of Fortune: 2nd Edition
Where the Wild Things Are
Where's Waldo? The Fantastic Journey
Whiplash
Whirl Tour
White Album 2: Shiawase no Mukougawa
White Album: Tsuzurareru Fuyu no Omoide
White Knight Chronicles II
White Knight Chronicles: Internatio

Zetta Hero Project: Unlosing Ranger vs. Darkdeath Evilman
Zettai Kaikyuu Gakuen: Eden with Roses and Phantasm
Zettai Meikyuu Grimm: Nanatsu no Kagi to Rakuen no Otome
Zettai Onkan Otoda Master
Zettai Zetsumei Toshi 3: Kowareyuku Machi to Kanojyo no Uta
ZhuZhu Babies
ZhuZhu Pets
ZhuZhu Pets 2: Featuring The Wild Bunch
ZhuZhu Pets: Featuring The Wild Bunch
ZhuZhu Pets: Quest for Zhu
ZhuZhu Puppies
Zill O'll Infinite Plus
Zillions of Enemy X: Zetsukai no Crusade
Zoboomafoo: Leapin' Lemurs!
Zoey 101: Field Trip Fiasco
Zoids Assault
Zoids Dash
Zoids: Battle Legends
Zoids: Legacy
Zombi Daisuki
ZombiU
Zombie Army Trilogy
Zombie Hunters 2
Zone of the Enders
Zone of the Enders HD Collection
Zone of the Enders: The 2nd Runner
Zone of the Enders: The Fist of Mars
Zoo Hospital
Zoo Keeper
Zoo Quest: Puzzle Fun!
Zoo Resort 3D
Zoo Tycoon
Zoo Tycoon (2013)
Zoo Tycoon 2
Zoo Tycoon 2 DS
Zoo Tycoon 2: Ultimate Collection
Zoo Tycoon DS
Zoo Tycoon: Complete Collection
Zoo Vet: Endangered Animals
ZooCube
Zo

In [7]:
print(data['platform'].unique())
print(data['year_of_release'].unique())
print(data['genre'].unique())
print(data['rating'].unique())

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
[2006. 1985. 2008. 2009. 1996. 1989. 1984. 2005. 1999. 2007. 2010. 2013.
 2004. 1990. 1988. 2002. 2001. 2011. 1998. 2015. 2012. 2014. 1992. 1997.
 1993. 1994. 1982. 2016. 2003. 1986. 2000.   nan 1995. 1991. 1981. 1987.
 1980. 1983.]
['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan]
['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']


In [8]:
for i in range(len(data['name'])):
    if np.isnan(data.loc[i,'year_of_release']) == True:
        for j in range(len(data['name'])):
            if (data.loc[i,'name'] == data.loc[j,'name']) and (np.isnan(data.loc[j,'year_of_release'])== False) and (data.loc[i,'platform'] == data.loc[j,'platform']):
                data.loc[i,'year_of_release'] = data.loc[j,'year_of_release']
                
                

In [9]:
data = data.dropna(subset=['name'])